<a href="https://colab.research.google.com/github/Shk-tech/my-colab-learning-material/blob/main/2507_korea_housing_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 데이터 수집..
 - 됐다고 가정

In [ ]:
from pathlib import Path
import pandas as pd
import tarfile
import urllib.request

def load_housing_data():
    tarball_path = Path("datasets/housing.tgz")
    if not tarball_path.is_file():
        Path("datasets").mkdir(parents=True, exist_ok=True)
        url = "https://github.com/ageron/data/raw/main/housing.tgz"
        urllib.request.urlretrieve(url, tarball_path)
        with tarfile.open(tarball_path) as housing_tarball:
            housing_tarball.extractall(path="datasets")
    return pd.read_csv(Path("datasets/housing/housing.csv"))

housing = load_housing_data()

In [ ]:
# 1. 데이터 탐색(study) - corr, 히스토그램, scatter....

#----전처리 과정-----
# 4. 결측치 처리
# 5. x-> numeric, categorical
# 5.1   numeric-> 특징 추가, scaling
# 5.2   categorical -> onehot-enc, 단순 수치 변환  (str->float)
# 5.3 둘을 합치면 전처리 끝.

#----머신러닝-----
# 2. split train - valid(test)
# 3. split x, y
# 6. model 선정, fit
# 7. 성능측정, 검증

In [ ]:
# 4. 결측치 처리...
housing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


,0
total_bedrooms,435.0
population,1166.0


In [ ]:
# 확인 결과 total_bedrooms 에 결측치 존재
# 중간값으로 채운다.
median = housing[['total_bedrooms', 'population']].median()#default-(axis=0)
median

df2 = housing.fillna(median)
# df2 = housing.copy()
# df2['total_bedrooms'] = df2['total_bedrooms'].fillna(median)
df2.isnull().any().any()

np.False_

In [ ]:
import numpy as np
num = df2.select_dtypes(np.number)
cat = df2.select_dtypes(object) # ??
# df2.columns
# cat = df2[['ocean_proximity']]
# num_cols = ['longitude', 'latitude', 'housing_median_age', 'total_rooms',
#        'total_bedrooms', 'population', 'households', 'median_income',
#        'median_house_value']
# df2[num_cols]

# num_cols = set(df2.columns) - set(['ocean_prolimity'])
# df2[list(num_cols)]

In [ ]:
# 스케일링 - Training 데이터만 가지고 하는게 정석...
mean = num.mean()
std = num.std()
num2 = (num - mean) / std
num2.std()

,0
longitude,1.0
latitude,1.0
housing_median_age,1.0
total_rooms,1.0
total_bedrooms,1.0
population,1.0
households,1.0
median_income,1.0
median_house_value,1.0


In [ ]:
# cat
cat2 = pd.get_dummies(cat)

In [ ]:
# 다시 통합... preprocessing 완성
prep = pd.concat([num2, cat2], axis=1)

In [ ]:
# 본격적인 머신러닝...
from sklearn.model_selection import train_test_split
y = prep[['median_house_value']]
x = prep.drop('median_house_value', axis=1)
test_size = int(len(x) * 0.2)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_size, shuffle=True)

In [ ]:
len(x_train), len(x_test)

(16512, 4128)

In [ ]:
# 오버피팅(overfitting, 과대적합)
# 오버피팅이란?
# - train에서는 잘 동작, test에서 오동작
#   - 일반성을 잃어버리는 것이다.
#   - train데이터에 우연히 나타난 특징을 일반적인 특징으로 오인 ()
# - 쓸모없는 특징을 중요한 특징으로 오인
# - 분석하기보다 샘플을 암기해버린다.
# - 디테일에 집착하여 전체적인 문맥을 잃어버리는 현상.
# 원인
# - 데이터의 개수가 적다.
# - 데이터의 품질이 나쁘다.
# - 모델이 데이터의 문맥을 잡아내는 능력이 없다.
# 해결
# -> 데이터의 수를 늘린다 (노이즈를 추가한다.)
# -> 데이터의 품질을 향상
# -> 모델 개선 (디테일 x 문맥 o)

# 확인
# -> test(valid)데이터로 확인
# -> 학습단계에서 test데이터는 감추어야 한다.
# -> train / (valid / test)
# ->  8 : 2 ,,,, 9 : 1

In [ ]:
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()
linreg.fit(x_train, y_train)
# linreg.fit(x_train.values, y_train.values) #numpy행렬이 들어간다
# N,F1 -> N, F2

# sklearn의 regressor
# fit함수에서 데이터만 제공하는 게 아니라 입력크기, 출력 크기도 정해준다.
x_train.values.shape
# (16512, 13)
#  N, 13크기를 입력으로 넣는다.
#  axis=0.. shape에서 맨 앞의 숫자 -> N (샘플의 개수)

y_train.values.shape
# (16512, 1)
# (N, 1) 크기를 출력으로 한다.


(16512, 1)

In [ ]:
# 학습 끝나고 활용할 때... fit에서 넣었던 shape대로 작동한다!!!!!
# fit(X, Y) 여기서 X에 뭘 넣었더라?? -> N, 13
# 그렇다면 predict할 때도 N, 13을 넣어야 한다.
# Y에 뭘 넣었더라?? -> N, 1
# 그렇다면 출력 결과도 N, 1이다.

# 1개의 데이터를 넣어서 1개를 출력
# (1, 13) -> (1, 1) ... (13,)
def unscale(x):
    return x * std['median_house_value'] + mean['median_house_value']
pred = linreg.predict(x_train.values) #(N, 1) <- fit(X, Y)에서 Y의 shape 그대로 출력
pred = unscale(pred)
pred

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


array([[ 48552.62114783],
       [330240.63025726],
       [335111.98555797],
       ...,
       [173648.05471787],
       [281731.90669979],
       [189013.70808054]])

In [ ]:
# training error (MSE)
# pred, y_train(true, target, ground-truth : 정답)
target = unscale(y_train.values)
mse = ((pred - target) ** 2).mean() #스케일링된 에러....
rmse = mse ** 0.5
rmse

np.float64(68953.83293944524)

In [ ]:
pred = linreg.predict(x_test)
pred = unscale(pred)
pred

array([[232891.85032896],
       [184388.60487079],
       [156840.40581424],
       ...,
       [306708.3855887 ],
       [207011.02660692],
       [183136.04167952]])

In [ ]:
target = unscale(y_test.values)
#mse
mse = ((pred - target) ** 2).mean()
rmse = mse ** 0.5
rmse

np.float64(67771.44017713127)

In [ ]:
#regressor
from sklearn.tree import DecisionTreeRegressor
treereg = DecisionTreeRegressor()
#fit...
treereg.fit(x_train, y_train)

#training rmse
#1. pred
pred = unscale(treereg.predict(x_train))
# #2. mse
target = unscale(y_train.values[:, 0])
mse = ((pred - target)**2).mean()
# #3. rmse
rmse = mse ** 0.5
rmse


np.float64(1.5951118817192326e-11)

In [ ]:
#1. pred
pred = unscale(treereg.predict(x_test))
# #2. mse
target = unscale(y_test.values[:, 0])
mse = ((pred - target)**2).mean()
# #3. rmse
rmse = mse ** 0.5
rmse

np.float64(67287.67345526075)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rfreg = RandomForestRegressor()
rfreg.fit(x_train, y_train)

/usr/local/lib/python3.11/dist-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestRegressor()

In [ ]:
#1. pred
pred = unscale(rfreg.predict(x_train))
#2. mse
target = unscale(y_train.values[:, 0])
mse = ((pred - target)**2).mean()
# #3. rmse
rmse = mse ** 0.5
rmse


np.float64(18392.30353228968)

In [ ]:
#1. pred
pred = unscale(rfreg.predict(x_test))
#2. mse
target = unscale(y_test.values[:, 0])
mse = ((pred - target)**2).mean()
# #3. rmse
rmse = mse ** 0.5
rmse


np.float64(47656.621966285085)